![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## Więcej niż MapReduce

In [1]:
#http://localhost:4040/jobs/
import findspark
findspark.init()

In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder \
    .appName("beyondMapReduce") \
    .getOrCreate()
sc = spark.sparkContext

## Akumulatory

Akumulatory to zmienne tylko do zapisu dla workerów, które służą do zliczania wystąpień.

In [3]:
acc = sc.accumulator(0)

In [4]:
rdd = sc.textFile("data/titus_andronicus.txt")

In [5]:
rdd.foreach(lambda x: acc.add(1))

In [6]:
# Wartość dostępna jest jako
acc.value

3255

In [7]:
# Akumulatory można opakowywać w funkję, jeżeli zachodzi taka potrzeba
def function_with_acc(x):
    acc.add(1)

In [8]:
rdd.foreach(function_with_acc)

In [9]:
acc.value

6510

In [35]:
# Zainicjujmy kolejny akumulator
acc2 = sc.accumulator(0)

In [36]:
# ... i związaną funkcję
def split_and_countspaces(x):
    acc2.add(x.count(" "))
    return x.split()

In [37]:
words = rdd.flatMap(split_and_countspaces)
words.take(10)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation']

### Zadania 

* Jaka jest wartość `acc2`?
* Pobierz 15 wyrazów: jaka jest teraz wartość `acc2`?
* Pobierz jeszcze raz i sprawdź wartość.
* Użyj `words.cache()`, pobierz 15 wyrazów i sprawdź `acc2.value`.
* Weż pierwsze 15 wyrazów i sprawdź `acc2.value`.

In [38]:
acc2.value

17

In [39]:
words.take(10)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation']

In [40]:
acc2.value

34

In [41]:
words.cache()
words.take(10)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation']

In [42]:
acc2.value

16059

## Broadcasts

Zmienne tylko do odczytu pozwalające przekazywać argumenty do zadań. 

In [43]:
# Zmiennych tych używa się do przekazywania argumentów; wartości te mogą być dość duże
br = sc.broadcast(["Marcus", "Titus", "hate", "die"])

In [44]:
br.value

['Marcus', 'Titus', 'hate', 'die']

In [45]:
def line_of_brothers(line):
    t = 0
    for word in br.value:
        if word in line:
            t += 1
    return t > 1

In [46]:
rdd.filter(line_of_brothers).take(5)

['    Let Marcus, Lucius, or thyself, old Titus,',
 "  TITUS. Marcus, my brother! 'Tis sad Titus calls."]

In [47]:
rdd.filter(line_of_brothers).takeSample(False, 5)

["  TITUS. Marcus, my brother! 'Tis sad Titus calls.",
 '    Let Marcus, Lucius, or thyself, old Titus,']

## Operacje na zbiorach

In [48]:
rdd1 = sc.parallelize(["cat", "dog", "python"])
rdd2 = sc.parallelize(["magpie", "python", "elephant"])

In [49]:
# Konkatenacja a nie suma zbiorów
rdd1.union(rdd2).collect()

['cat', 'dog', 'python', 'magpie', 'python', 'elephant']

### Zadanie

* Żeby otrzymać prawdziwą sumę zbiorów z `union` użyj `.distinct()`.

In [51]:
rdd1.union(rdd2).distinct().collect()

['python', 'elephant', 'magpie', 'dog', 'cat']

In [50]:
# Część wspólna zbiorów
rdd1.intersection(rdd2).collect()

['python']

## Złączenie zbiorów

Łączenie zbiorów (join) działa podobnie jak w SQL ale odbywa się na parze klucz wartość w formie `(key, value)`.

In [52]:
species = sc.parallelize([("Max", "dog"), ("Sasha", "cat"), ("Alice", "snake")])
ages = sc.parallelize([("Alice", 3), ("Max", 11), ("Ulrich", 2), ("Karl", 31)])

In [53]:
# inner join
species.join(ages) \
  .collect()

[('Max', ('dog', 11)), ('Alice', ('snake', 3))]

### Zadanie

* Wypisz zbiór wierząt ze znanym wiekiem.

In [ ]:
# Podpowiedź
z = ("a", ("b", "c"))
print(z[1])
print(z[1][0])

In [58]:
species.join(ages).map(lambda x: x[1][0]).distinct().collect()

['dog', 'snake']

In [ ]:
#species.join(ages).mapValues()

In [ ]:
# left outer join
species.leftOuterJoin(ages) \
  .collect()

In [ ]:
# right outer join
species.rightOuterJoin(ages) \
  .collect()

In [ ]:
# full outer join
species.fullOuterJoin(ages) \
  .collect()

In [ ]:
# iloczyn kartezjański
species.cartesian(ages) \
  .collect()

### Zadanie

* ★ Zamień produkt kartezjański w wewnętrzne złączenie (inner join).
* ★ Wypisz wszystkie możliwe pary zwierząt, np. `[("Max", "Alice"), ... ("Ulrich", "Karl")]`.

## Grupowanie (po kluczu)

In [59]:
cities = sc.parallelize(["Paris, France", "Lyon, France", "Warsaw, Poland", "London, United Kingdom"])

In [60]:
cities \
  .groupBy(lambda x: x.split(", ")[1]) \
  .map(lambda x: (x[0], list(x[1]))) \
  .collect()

[('Poland', ['Warsaw, Poland']),
 ('United Kingdom', ['London, United Kingdom']),
 ('France', ['Paris, France', 'Lyon, France'])]

### Zadanie

* Zamiast `.map(lambda x: (x[0], list(x[1])))` użyj `.mapValues(some_function)`. 

In [61]:
# Podpowiedź
sc.parallelize([("a", 3), ("b", -2)]) \
  .mapValues(lambda x: x*x) \
  .collect()

[('a', 9), ('b', 4)]

### Więcej zadań

Wracamy do Titus Andronicus:

* Wyczyść dane przez usuniecie niechcianych znaków
* Policz i wyświetl 20 najczęstrzych:
    * słów zaczynających się zawsze wielką literą,
    * słów nigdy nie zaczynających się od wielkiej litery.
* Policz częstotliwość (i.e. $p_{word}=n_{word}/n$ wszystkich słów (ignorując wielkość liter).
* Pobierz słownik z [tej strony](http://www.math.sjsu.edu/~foster/dictionary.txt); policz ile słów ze słownika występuje w dziele (słownik ma tylko wyrazy o małych literach).
* ★ Pobierz dzieło nie napisane przez Shakespeare [Gutenberg Project](http://www.gutenberg.org/); znajdź słowa charakterystyczne dla Shakespeare'a, tj. które nie występują w innym dziele.
* ★ Dla wszystkich słów w lini policz

$$\frac{p_{pair}}{p_{word1}p_{word2}} = \frac{n_{pair} n}{n_{word1}n_{word2}},$$

tylko dla par występujących więcej niż 5 razy; pokaż 20 najwyższych wartości.

In [62]:
rdd.take(10)

['',
 'This Etext file is presented by Project Gutenberg, in',
 'cooperation with World Library, Inc., from their Library of the',
 'Future and Shakespeare CDROMS.  Project Gutenberg often releases',
 'Etexts that are NOT placed in the Public Domain!!',
 '',
 '*This Etext has certain copyright implications you should read!*',
 '',
 '<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM',
 'SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS']